<a href="https://colab.research.google.com/github/weihanchen/google-colab-python-learn/blob/main/jupyter-examples/kd_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 安裝必要套件
- [x] [yfinance](https://pypi.org/project/yfinance/): 獲取Yahoo Finance API的小幫手
- [x] [backtesting](https://kernc.github.io/backtesting.py/): 回測工具
- [x] [ta-lib](https://github.com/TA-Lib/ta-lib-python): 技術指標工具

P.S 此安裝步驟會運行一段時間， 主要是因為Ta-Lib的安裝方式比較特殊，需要重新Compile，約耗時3分鐘，可以喝個咖啡再回來看結果 😋

In [ ]:
# 安裝必要套件
!pip install yfinance backtesting

# 安裝Ta-Lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

# 共用參數設定
- 股票代號
- 開始時間
- 結束時間
- 初始現金
- 手續費

# 下載個股資訊
## 參數
- 股票代號。
- 開始時間。
- 結束時間。

In [37]:
import yfinance as yf
import pandas as pd
from pandas_datareader import data
from datetime import datetime

################################# 請視需求帶入參數 ######################################
# 股票代號
stock_code = '0050'

# 開始時間
start_date = datetime(2014, 7, 25)

# 結束時間
end_date = datetime(2022, 7, 25) 
###########################################################################################

# 獲取股票資料
yf.pdr_override() 

# 從yahoo資料源獲取台股代號的datafram
df = data.get_data_yahoo(stock_code+'.TW', start_date, end_date) 

df.tail(10)

# filename = f'./data/{stock_code}.csv' #以股票名稱命名檔案，放在data資料夾下面

# df.to_csv(filename) #將df轉成CSV保存

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-07-11,113.650002,113.699997,111.849998,112.250000,112.250000,7572953
2022-07-12,110.949997,110.949997,109.150002,109.550003,109.550003,11133444
2022-07-13,113.000000,113.650002,112.599998,113.449997,113.449997,24824897
2022-07-14,112.699997,114.250000,111.800003,114.000000,114.000000,14750428
2022-07-15,114.599998,115.650002,113.750000,115.500000,115.500000,18555312
2022-07-18,114.650002,114.849998,113.949997,114.199997,114.199997,22207344
2022-07-19,113.949997,114.650002,113.599998,114.300003,114.300003,13251365
2022-07-20,115.849998,116.199997,114.699997,115.050003,115.050003,16836288
2022-07-21,115.400002,116.349998,115.000000,116.349998,116.349998,8803270


In [38]:
from backtesting import Backtest, Strategy 

from backtesting.lib import crossover
from backtesting.test import SMA

from talib import abstract

import pandas as pd


# bypass data in Strategy
def I_bypass(data): 
    return data

class KDCross(Strategy): 
    lower_bound = 20  
    upper_bound = 80  

    def init(self):
        print(self.data)
        self.k = self.I(I_bypass, self.data.slowk) 
        self.d = self.I(I_bypass, self.data.slowd)

    def next(self):
        if crossover(self.k, self.d) and self.k<self.lower_bound and self.d<self.lower_bound and not self.position: #long position
            self.buy() 
        elif crossover(self.d, self.k) and self.k>self.upper_bound and self.d>self.upper_bound: 
            if self.position and self.position.is_long:
                self.position.close()

# CSV檔案中若有缺漏，會使用內插法自動補值，不一定需要的功能
df = df.interpolate() 

# 將索引欄資料轉換成pandas的時間格式，backtesting才有辦法排序
df.index = pd.to_datetime(df.index) 

#calculate KD signal with talib
df_tmp = df
df_tmp.rename(columns = {'High':'high', 'Low':'low','Adj Close':'close','Close':'non_adj close'}, inplace = True) #rename for talib
kd = abstract.STOCH(df_tmp)
kd.index=df_tmp.index
fnl_df = df_tmp.join(kd).dropna() #merge two data frame
fnl_df.rename(columns = {'high':'High', 'low':'Low','close':'Close'}, inplace = True) #rename column name for backtest

# 資料來源, 交易策略, 現金, 手續費(%)
test = Backtest(fnl_df, KDCross, cash=10000, commission=.004)


# 執行回測程式並存到result中
result = test.run()

print(result)




<Data i=1948 (2022-07-22 00:00:00) Open=116.5999984741211, High=117.19999694824219, Low=116.30000305175781, non_adj close=116.80000305175781, Close=116.80000305175781, Volume=15614914.0, slowk=87.58426418207061, slowd=79.78252097827898>
Start                     2014-08-06 00:00:00
End                       2022-07-22 00:00:00
Duration                   2907 days 00:00:00
Exposure Time [%]                    31.65726
Equity Final [$]                  9325.253538
Equity Peak [$]                  10846.046579
Return [%]                          -6.747465
Buy & Hold Return [%]               77.777791
Return (Ann.) [%]                   -0.899189
Volatility (Ann.) [%]               11.080167
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -34.694656
Avg. Drawdown [%]                  -13.777616
Max. Drawdown Duration     2583 days 00:00:00
Avg. Drawdown Duration     